In [1]:
import os
os.chdir( "../")

In [1]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

rp3_study = optuna.load_study(study_name="RP3_Hybrid_last", storage=storage)
p3_study = optuna.load_study(study_name="P3_Hybrid_last", storage=storage)
slim_study = optuna.load_study(study_name="SLIM_Hybrid_last", storage=storage)
knn_study = optuna.load_study(study_name="KNN_Hybrid_last", storage=storage)

hybrid_study = optuna.load_study(study_name="LINEAR_TOPPOP_Hybrid_last", storage=storage)

In [4]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [5]:
target_users = pd.read_csv("Dataset/data_target_users_test.csv")
target_users.columns = ["UserID"]

In [6]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [10]:
from Recommenders.Recommender_import_list import *
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.Hybrid.GeneralizedLinearHybridRecommenderCold import GeneralizedLinearHybridRecommenderCold
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender

2024-01-09 23:33:41.229654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-09 23:33:41.229686: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
alphas = [hybrid_study.best_params['w_slim'], hybrid_study.best_params['w_rp3'], hybrid_study.best_params['w_p3'], hybrid_study.best_params['w_knn'], hybrid_study.best_params['w_top']]



{'alpha': 0.0010318988432401724, 'l1_ratio': 0.11924623884276588, 'topK': 8529}

In [11]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_all)
slim.fit(workers=8, **slim_study.best_params)

rp3 = RP3betaRecommender(urm_all)
rp3.fit(**rp3_study.best_params)

p3 = P3alphaRecommender(urm_all)
p3.fit(**p3_study.best_params)

knn = ItemKNNCFRecommender(urm_all)
knn.fit(**knn_study.best_params)

top = TopPop(urm_all)
top.fit()


RP3betaRecommender: Similarity column 22222 (100.0%), 4536.92 column/sec. Elapsed time 4.90 sec


In [11]:
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender

hybrid = GeneralizedLinearHybridRecommender(urm_all, recommenders=[slim, rp3, p3, knn, top])
hybrid.fit(alphas=alphas)

In [12]:
item_popularity_encoded = np.ediff1d(urm_all.tocsc().indptr)
item_popularity_encoded = np.sort(item_popularity_encoded)

tar_users = target_users["UserID"].astype(int)
topPop_encoded = item_popularity_encoded[-10:]

submission = []

print(np.unique(df["UserID"].values))

for index, user in enumerate(tar_users):
    if (user not in df["UserID"].values):
        item_list_encoded = topPop_encoded
    else:
        item_list_encoded = hybrid.recommend(user2user_encoded[user])[:10]
    item_list = []
    for item_encoded in item_list_encoded:
        item_list.append(item_encoded2item[item_encoded])
    submission.append((user, item_list))


def write_submission(submissions):
    with open("./submission_hybrid_combo_script.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
            
write_submission(submission)

[    1     2     3 ... 13022 13023 13024]
